In [1]:
%matplotlib inline


import pandas as pd
import timeit
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import tree
import xgboost as xgb


//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
s_train_path = "anon/s_train.csv"
s_train = pd.read_csv(s_train_path,sep='\t')
s_train = s_train.drop(s_train.columns[0],1)

In [3]:
s_train.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,11,9,109,12,416,1273,0,19,19,0,12,3580,19,-1,1
1,11,9,109,12,416,1226,0,30,30,0,12,3782,4,-1,1
2,11,9,109,12,416,1274,0,33,33,0,12,3165,19,-1,1
3,11,9,144,12,281,959,0,30,30,0,12,3404,29,-1,0
4,11,9,109,12,416,1273,0,19,19,0,12,3406,3,-1,1


In [4]:
def create_dummies(s_train):    
    start = timeit.default_timer()
    columns = list(s_train.columns.values)
    for column in columns:
        if column != "V6" and column !="V12" and column != "V15":
            column_dummy = pd.get_dummies(s_train[column],prefix=column)
            s_train = s_train.drop(column,1)
            s_train = s_train.join(column_dummy)
            print("Finished processing "  , column, ", Time used: {:03.1f} s".format(timeit.default_timer()-start))
    V6_dummy = pd.get_dummies(s_train["V6"],prefix="V6")
    s_train = s_train.drop("V6",1)
    s_train = s_train.join(V6_dummy)
    print("Finished processing  V6 , Time used: {:03.1f} s".format(timeit.default_timer()-start))
    #V12_dummy = pd.get_dummies(s_train["V12"],prefix="V12")
    #s_train = s_train.drop("V12",1)
    #s_train = s_train.join(V12_dummy)
    #print("Finished processing  V12 , Time used: {:03.1f} s ".format(timeit.default_timer()-start))
    s_train = s_train.rename(index=str, columns={"V15": "label"})
    return s_train

In [5]:
s_train_dummy = create_dummies(s_train)

Finished processing  V1 , Time used: 0.2 s
Finished processing  V2 , Time used: 0.3 s
Finished processing  V3 , Time used: 1.3 s
Finished processing  V4 , Time used: 2.3 s
Finished processing  V5 , Time used: 5.4 s
Finished processing  V7 , Time used: 8.8 s
Finished processing  V8 , Time used: 12.4 s
Finished processing  V9 , Time used: 15.7 s
Finished processing  V10 , Time used: 20.0 s
Finished processing  V11 , Time used: 24.0 s
Finished processing  V13 , Time used: 28.9 s
Finished processing  V14 , Time used: 51.1 s
Finished processing  V6 , Time used: 151.3 s


In [6]:
Y = s_train_dummy["label"]
X = s_train_dummy.drop("label",1).drop("V12",1)

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

In [8]:
X_train.head()

,V1_0,V1_1,V1_2,V1_3,V1_4,V1_5,V1_6,V1_7,V1_8,V1_9,...,V6_4747,V6_4748,V6_4749,V6_4750,V6_4751,V6_4752,V6_4753,V6_4754,V6_4755,V6_4756
61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.values
Y_test = Y_test.values



In [10]:
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)

dtrain.save_binary('train.buffer')
dtest.save_binary('test.buffer')


In [11]:
print(s_train_dummy.shape,X.shape,Y.shape)

(139512, 7691) (139512, 7689) (139512,)


In [12]:
print("completed create file")

completed create file


In [13]:
X.head()

,V1_0,V1_1,V1_2,V1_3,V1_4,V1_5,V1_6,V1_7,V1_8,V1_9,...,V6_4747,V6_4748,V6_4749,V6_4750,V6_4751,V6_4752,V6_4753,V6_4754,V6_4755,V6_4756
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
